In [1]:
import json
fname = "../xcode/Release/response_tracks.json"
with open(fname,"r") as f:
    data = json.load(f)

In [2]:
deadtracks = filter(lambda x: x["numSamples"] > 5, data["deadTracks"]);
livetracks = filter(lambda x: x["numSamples"] > 5, data["liveTracks"]); 
tracks = deadtracks
tracks.extend(livetracks)
len(tracks)

876

In [3]:
%matplotlib qt
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.lines as lines
import matplotlib.patches as patches
def simpleaxis(ax):
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.get_xaxis().tick_bottom()
    ax.get_yaxis().tick_left()

In [5]:
len(tracks)

876

In [14]:
Time = 120.0
Frames = 870.0

plt.rc('text', usetex=True)
plt.rc('font', family='serif',size=5,weight="bold")
plt.style.use("bmh")
fig,ax = plt.subplots()
plt.xlim([0,Time])
plt.ylim([-400,400])

ax.add_line(lines.Line2D([30,30],  [-400,400],linewidth=2,linestyle="--",color='gray'))
ax.add_line(lines.Line2D([60,60],  [-400,400],linewidth=2,linestyle="--",color='gray'))
ax.add_line(lines.Line2D([90,90],  [-400,400],linewidth=2,linestyle="--",color='gray'))


for t in tracks:
    frames = [ x["frame"]/Frames * Time for x in t["samples"]]
    center_x = [ x["rect"][0] - t["samples"][0]["rect"][0] for x in t["samples"]]    
    #plt.hold(True)
    plt.plot(frames,center_x,color='darksalmon',linewidth=1)

plt.xticks((0,30,60,90,120))
plt.yticks((-300,0,300))
ax.grid(False)
simpleaxis(ax)
plt.savefig("/Users/zhossain/Dropbox/151 Paper Euglena Online Zahid/Figures/Automonitor/Trace X.pdf",transparent=True)
plt.close(fig)
#plt.title('Relative X position of Euglena Vs Time')
#plt.xlabel('Time (sec)')
#plt.ylabel('Relative X position (pixels)'



In [15]:
fig,ax = plt.subplots()
plt.xlim([0,Time])
plt.ylim([-400,400])
# ax.add_patch(patches.Rectangle((15,-400),15,800,alpha=0.1))
# ax.add_patch(patches.Rectangle((45,-400),15,800,alpha=0.1))
# ax.add_patch(patches.Rectangle((75,-400),15,800,alpha=0.1))
# ax.add_patch(patches.Rectangle((105,-400),15,800,alpha=0.1))

ax.add_line(lines.Line2D([30,30],  [-400,400],linewidth=2,linestyle="--",color='gray'))
ax.add_line(lines.Line2D([60,60],  [-400,400],linewidth=2,linestyle="--",color='gray'))
ax.add_line(lines.Line2D([90,90],  [-400,400],linewidth=2,linestyle="--",color='gray'))
    
for t in tracks:
    frames = [ x["frame"]/Frames * Time for x in t["samples"]]
    center_y = [ t["samples"][0]["rect"][1] - x["rect"][1]  for x in t["samples"]]    
    #plt.hold(True)
    plt.plot(frames,center_y,color='teal',linewidth=1)


ax.grid(False)
simpleaxis(ax)
plt.xticks((0,30,60,90,120))
plt.yticks((-300,0,300))
plt.savefig("/Users/zhossain/Dropbox/151 Paper Euglena Online Zahid/Figures/Automonitor/Trace Y.pdf",transparent=True)
plt.close(fig)
#plt.title('Relative Y position of Euglena Vs Time')    
#plt.xlabel('Time (sec)')
#plt.ylabel('Relative Y position (pixels)')


In [ ]:
import math
# def extractPositionsTimeSlot(start,end, fps, tracks):
#     startFrame = math.floor(fps * start)
#     endFrame = math.ceil(fps * end)
#     samples [ for s in t["samples"] for t in tracks if s["frame"] >= startFrame && s["frame"] <= endFrame ]
            
def truncatedTracks(start,end,fps,tracks):
    startFrame = math.floor(fps * start)
    endFrame = math.floor(fps * end)
    filteredTracks = []
    for t in tracks:
        if t["startFrame"] <= endFrame:
            trackSamples = []
            for s in t["samples"]:
                if s["frame"] >= startFrame and s["frame"] < endFrame:
                    trackSamples.append(s)
            filteredTracks.append( trackSamples );
    return filteredTracks    

In [ ]:
def computeVelocities(tracks,fps):
    xlist = [];
    ylist = [];
    for t in tracks:
        for i in xrange(1,len(t)):
            rect2 = t[i]["rect"]
            frame2 = t[i]["frame"]
            rect1 = t[i-1]["rect"]
            frame1 = t[i-1]['frame']
            
            vx = (rect2[0] - rect1[0]) * fps / float( frame2 - frame1 )
            vy = -(rect2[1] - rect1[1]) * fps / float( frame2 - frame1 )
            xlist.append(vx)
            ylist.append(vy)
    return xlist,ylist

In [ ]:
#plt.rc('text', usetex=True)
plt.rc('font', family='serif',size=40,weight="bold")
for i in range(4):
    start = i*30
    end = start + 30
    tTracks = truncatedTracks(start,end,Frames/Time,tracks)
    xlist,ylist = computeVelocities(tTracks,Frames/Time)
    speed = np.sqrt(np.array(xlist)**2 + np.array(ylist)**2)
    # plt.hist(ylist,100,normed=1)
    fig,ax = plt.subplots()
    bins = math.floor(math.sqrt(len(xlist)))
    _ = plt.hist(xlist,bins,normed=1,color='gray',alpha=0.5)
    plt.xticks((-60,0,60))    
    
    
        
    meanx = np.mean(xlist)
    medianx = np.mean(xlist)

    ax.add_line(lines.Line2D([0,0],  [0,0.03],linewidth=4,linestyle='--',color='black'))
    ax.add_line(lines.Line2D([medianx,medianx],  [0,0.03],linewidth=4,color='red'))
    ax.grid(False)
    ax.set_axis_bgcolor('white')
    
    plt.setp(ax.get_yticklabels(), visible=False)    #plt.hist(speed,bins,normed=1)
    ax.yaxis.set_tick_params(size=0)
    

    plt.xlim(-100,100)
    plt.ylim(0,0.03)
    #plt.title('Speed distribution during $%d-%d$ sec, (Mean=$%2.1f$)'%(start,end,meanx))
    #plt.xlabel('Velocities (pixels/sec)')
    
    plt.savefig("/Users/zhossain/Dropbox/151 Paper Euglena Online Zahid/Figures/Automonitor/vx_%d_%d.pdf"%(start,end),transparent=True)
    plt.close(fig)
   

    fig,ax = plt.subplots()
    _ = plt.hist(ylist,bins,normed=1,color='gray',alpha=0.5)
    plt.xticks((-60,0,60)) 

    meany = np.mean(ylist)
    mediany = np.mean(ylist)

    ax.add_line(lines.Line2D([0,0],  [0,0.03],linewidth=4,linestyle='--',color='black'))
    ax.add_line(lines.Line2D([mediany,mediany],  [0,0.03],linewidth=4,color='red'))
    ax.grid(False)
    ax.set_axis_bgcolor('white')
    
    plt.setp(ax.get_yticklabels(), visible=False)    #plt.hist(speed,bins,normed=1)
    ax.yaxis.set_tick_params(size=0)
    
    plt.xlim(-100,100)
    plt.ylim(0,0.03)
    #plt.title(' Speed distribution during $%d-%d$ sec, (Mean=$%2.1f$)'%(start,end,meany))
    #plt.xlabel('Velocities (pixels/sec)')
    plt.savefig("/Users/zhossain/Dropbox/151 Paper Euglena Online Zahid/Figures/Automonitor/vy_%d_%d.pdf"%(start,end),transparent=True)
    plt.close(fig)

In [ ]:
endFrame